In [1]:
#!pip --quiet install transformers datasets torch

In [32]:
import transformers
import datasets
from transformers.pipelines.pt_utils import KeyDataset
from pprint import PrettyPrinter
from tqdm.auto import tqdm
import gzip

pprint = PrettyPrinter(compact=True).pprint

In [ ]:
data=[]
with gzip.open("sorted_qa_binary_parsebank.tsv.gz", 'rt') as f:
    for line in f:
        line=line.rstrip("\n")
        cols=line.split("\t")
        data.append(cols)

texts = [text[2] for text in data]
ids = [text[0] for text in data]

In [3]:
def load_dataset(name, files):
    dset = datasets.load_dataset(
        "csv",
        data_files={name: files},
        delimiter="\t",
        split=name, # so that it returns a dataset instead of datasetdict
        column_names=['id', 'label', 'text', 'prob1', 'prob2'],
        features=datasets.Features({    # Here we tell how to interpret the attributes
        "text":datasets.Value("string"),
        "label":datasets.ClassLabel(names=["QA_NEW", "NOT_QA"]),
        "id":datasets.Value("string"),
        "prob1":datasets.Value("float"),
        "prob2":datasets.Value("float"),
         }))
    return dset

In [39]:
dataset = load_dataset("train", "qa-sorted/sorted_qa_binary_parsebank.tsv.gz")

Found cached dataset csv (/users/annieske/.cache/huggingface/datasets/csv/default-b202921bd9758045/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [40]:
def replace_n(example):
  example["text"] = example["text"].replace("\n", " ")
  example["text"] = example["text"].replace("\\n", " ")

  return example

dataset = dataset.map(replace_n)

Loading cached processed dataset at /users/annieske/.cache/huggingface/datasets/csv/default-b202921bd9758045/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-64a1815df4a04ae0.arrow


In [61]:
question_classifier = transformers.pipeline("ner", model="../models/token-qa/question_model2", tokenizer="xlm-roberta-base", aggregation_strategy="simple", ignore_labels=[""],device=0) # make ignore labels empty to not ignore O
# simple aggregation has been best so far?
 #stride=50, 

answer_classifier = transformers.pipeline("ner", model="../models/token-qa/answer_model", tokenizer="xlm-roberta-base", aggregation_strategy="simple", ignore_labels=[""],device=0)

token_classifier = transformers.pipeline("ner", model="../models/token-qa/joined_model", tokenizer="xlm-roberta-base", aggregation_strategy="simple", ignore_labels=[""],device=0)

In [62]:
questions_data = question_classifier(KeyDataset(dataset, "text")) #texts instead of keydataset
answers_data = answer_classifier(KeyDataset(dataset, "text"))
tokens = token_classifier(KeyDataset(dataset, "text"))

In [42]:
for idx, extracted_entities in enumerate(questions):
    print("Original text:\n{}".format(dataset[idx]["text"]))
    print("Original id:\n{}".format(dataset[idx]["id"]))
    print("Extracted entities:")
    for entity in extracted_entities:
        print(entity)

Original text:
Olet täällä  31.1.2011  Kysymys  Terve, semmoista olisin kysynyt, että mitä tarkoittaa aivopoimut ja mihinkä se mahdollisesti vaikuttaa? Minun 4,5v tyttöni aivot on kuvattu ja lääkäri sanoi että mahdollisesti aivopoimuja voi olla liian vähän. Puheterapeutille oli lähete, mutta hänen mielestään artikuloi hyvin ja muutenkin ok..ei tarvitse sitä... Kaiken tämän ruljanssin takana on pieni pää tytölläni, jota on tutkittu kohta jo 3,5 vuotta ja selkoa ei ole tullut. Hänellä on todettu mikrokefalia, pään ympärys on tällä hetkellä 42,3 cm...että osaisitteko selittää niistä poimuista ja mihin ne mahdollisesti vaikuttaa? Lääkäri epäili että voi olla vaikeampaa oppia jotain asioita. Mutta olisi mukavaa jos voisitte jotain vastailla.  Vastaus  Hyvä kysyjä,  Noihin tutkimustuloksiin ei pidä tuijottaa. Pääasiahan on miten tyttärenne kehittyy. Jolleivät vanhemmat eivätkä edes puheterapeutti ole huomanneet mitään kuntoutettavaa, voitte mielestäni unohtaa löydökset. Jos jotain poikkeavaa

Original text:
Viesti  1/29 Kysymys Kisa  Laitan joka toinen päivä kysymyksen eri aihepiiriltä joita on 3 vastaus aikaa on siis noin 2 pvä.  Vko. 1-2 = musiikkki (elikkä vko.1 on se milloin tulee ensimmäinen kysymys.)  Vko. 3-4 Urheilu  Vko. 5-6 Ruoka  Oikeasta vastauksesta saa 100 pistettä ensimmäinen oikein vastaaja saa kuitenkin aina 50 pistettä enemmän eli 150 pistettä.  Kun kaikki aihepiirit on käyty katsotaan 6 eniten pisteitä saanutta ja loput kuusi vähiten pisteitä saanutta putoaa kilpailusta. Kuusi parasta pääsee finaaliin. Pisteet jäävät finaaliin eli alku kisan parhaalla on parhaat mahdollisuudet.  Finaalissa kysyn joka toinen päivä kolmen viikon ajan jonkun kysymyksen eikä siitä kerrota aihepiiriä eikä aihepiiri ole pakosti mikään noista kolmesta. Finaalin ajan pisteet jakautuvat samalla lailla kuin alussa.  Kisan aikana annetaan tarvittaessa lisä ohjeita. Ilmoittautuminen alkaa nyt ja kisaan mahtuu 12 henkilöä  1. Se on sun tekemäsi (mainostusta) 2. Se ei ole liikkunut 3 k

Original text:
Selaa kategorioita  Minulla todettiin kondylooma  Kysymys  Hei . Olen nuori mies ja harrastin suojaamaton yhdyntää anaaliteitse. Lääkäri totesi ihan silmäilemällä että minulla on peräaukossa kondylooma? eikö siihen pitäisi mitään testiä tehdä että se varmasti varmistuu kondyloomaksi ? Mutta lääkärini oli vielä ulkomaalainen eli hän ei niin hyvin puhunu suomea minkä takia minulla jäi nytten hyvin epäselväksi tämä hoito juttu. Tilaanko uudestaan lääkärille ajan että saan jonkinlaisen käsityksen tähän. Sain wartec voidetta mutta se tuntuu niin hankalalta laittaa tuonne jos sitä ei saa vielä mennä terveeseen ihoon? Kuuria en ole vielä uskaltanut alottaa tämän epätietoisuuden takia.  Vastaus  Hei! Kondylooma voidaan todeta silmämääräisesti ja olet saanut siihen lääkäriltäsi oikeanlaisen hoidon. Valitettavasti peräaukolle sekä myös emättimen alueelle on itse hankalaa annostella liuosta tai voidetta. Lattialle asetetun peilin avulla se sujuu parhaiten. Hankalasti hoidettavissa 

Original text:
Ketonen vastaa  Öljyjen sekoitus  Kysymys:  Voiko 15-20 vuotiaaseen japsimoottoriin (200-300tkm ajettuun) käyttää sekaisin täyssynteettisiä 5W-30, 5W-40 ja 5W-50 öljyjä? Eli kysymys tarkoittaa siis sitä, kun kannujen pohjille on jäänyt eri öljynvaihtojen yhteydessä tuotetta. Jotkut väittää, ettei voi sekoittaa ja itse olen ajanut viimeiset 5 vuotta tällä periaatteella enkä ole ongelmia huomannut. Mitä sanoo asiantuntija? Tiedän, että kysymykseeni on vastaus olemassa jo tuossa edellä, mutta itse en näe mitään syytä miksei voisi sekoittaa öljyjä keskenään.  Tapio Ketonen:  Periaatteessa moottoriöljyjen API-luokitukseen kuuluu vaatimus eri merkkisten öljyjen sekoitettavuudesta, joten eiköhän homma toimi myös teoriassa, kun kerran olet sitä soveltanut käytäntöönkin.  Noissa uusimmissa eurooppalaisissa moottoreissa pitää olla vähän tarkempana, niissä kun on tiukemmat vaatimukset öljylaaduille. Esimerkiksi Fordin moottorit vaativat ACEA A1/B1-luokitusten mukaista öljyä. Valmis

Original text:
Vilkas sikiö (jatkoa)  Kysymys  Kävin tänään uusintaultrassa 21+6vko. Sikiön painoarvio oli 0,450 g ja päämitta vastasi viikkoja 22 ja raajat viikkoa 20. Kystaa ei näkynyt? Sovittiin raskauden jatkumisesta. Lapsivettä oli hiukan enemmän kuin normaalisti, napavirtaukset kaikki ok. Mistään löydöksistä ei mainittu enää? kasvot ok ja nenäkin ok. Ihmettelen enää tuota pään asiaa mansikkamaista, kun tänään sano sama lääkäri ettei ole mitään epämuodostumia??  Vastaus  Mansikkamainen pään muoto tarkoittaa, että pää on takaraivolta litteä ja siten mitta taka-eturaivo on tavallista lyhyempi. Tällainen pään muoto yhdistetään esim. 18-trisomiaan (yleensä muitakin löydöksiä).  Hieman tuntuu oudolta, että aikaisemmin oli runsaastikin erilaisia poikkevia löydöksiä (2 napasuonta, lyhyet luut, pleksuskysta jne) ja nyt siis kaikki normaalia. En pysty sanomaan mitään selitystä "kadonneille poikkeavuuksille". Kyllä asiaan pitää tutkineen lääkärin ottaa kantaa.  Kysymyksestä ei käy ilmi pitk

Original text:
Synnytys ja näkökyky  Kysymys  Hei!Olen 15vk:lla raskaana ja minua on huolettanut voinko synnyttää ns.normaalisti alateitse koska minulla on aika heikko näkö(13,75 diopteria)ja näkö on ollut huono 4-vuotiaasta lähtien ja huononee hiljakseen edelleen. Mitään silmäsairauksia minulla ei ilm.ole, taittovirhe ja likinäköisyys vain voimistuvat koko ajan. Muutamia vuosia sitten tehdyssä silmätarkastuksessa sanottiin etten saisi harrastaa mitään missä päähän voi kohdistua paineita(esim:painonnosto)ja äkkinäisiä repäiseviä liikkeitä tulisi varoa verkkokalvon repeämisen vaaran takia. Tämä on eka lapseni ja ymmärtääkseni synnytys on valtava ponnistusten sarja ja toisilta on katkeillut silmistä jopa pieniä verisuonia.. Kävin muutama päivä sitten uudelleen silmälääkärillä mutta hän ei suostunut ottamaan kantaa asiaan, jutteli vain ympäripyöreitä. Pitäisikö käydä toisella lääkärillä? En haluaisi menettää näköäni synnytyksessä..  Vastaus  Hyvä kysyjä, Kunkin synnyttäjän kohdalla synnyt

Original text:
Täysin koneellinen ilmanvaihto  Kysymykseni koskee ilmanvaihto suunnitelmaa, jonka mukaan tehdyn säädön jälkeen asuntomme on ilmeisesti liian alipaineinen koska iv-kone vetää korvausilman takan hormin kautta sisälle (molemmat savupellit ovat kiinni) taloon jonka seurauksena on epämielyttävä savunhaju asunnossa. Henkilö joka kävi säätämässa ko järjestelmän kehoitti avaamaan jokaista tuloventtiiliä n. 3kierrosta jotta saadaan asuntoa hieman ylipaineisemmaksi. Asia korjaantui hieman, mutta kun avaa khh:n oven(yhteydessä pesutiloihin) niin jälleen tulee haju sisään(myös keskuspölkkäri ja liesituuletin aiheuttavat saman ongelman). Siis olisiko laskelmat mahdolliseti päin prinkkalaa ja jos on niin mistä voisi saada asiaan korjauksen? Suunittelijaltako? Mitä muuta korjaavaa toimenpidettä mahdollisesti olemassa? Vai saako säätöjä muuttamalla asian kuntoon? Piippu on muurattu kahdella savupellillä varustettu ilman hattua(toistaiseksi)oleva. Kysymys 2. Kuinka monen kerroksen välei

Original text:
Kysymys Mikä on rakennekynsi?  Vastaus  Se on luonnonkynnen päälle rakennettava kynsi. Oli se sitten akryyli, geeli, lasikuitu tai silkki. Kaikilla näillä peitetään luonnonkynsi ja silloin se on rakennekynsi. Kuulee puhuttavan , että kynnen vahvistus ei olisi rakennekynsi. Sekin sitä on, koska oma kynsi peitetään jollain menetelmällä vahvistaen luonnon kynttä.  Kysymys Akryylin ja Geelin ero?  Vastaus  Kysymys Onko akryyli myrkyllistä?  Vastaus  Akryyli ei ole myrkyllistä. Akryylissä oleva haju on vahva, mutta ei myrkyllinen. Akryylistä tehdään esimerkiksi tekoniveliä ja hammaspaikkoja.  Vastaus  - Rakennekynsi ei aiheuta kynsisientä, vaan kynnen ja massan väliin jäänyt bakteeri tai kosteus. Ammattilainen puhdistaa kynnen oikein (ns.kuivataan), jotta bakteerit kuolevat kynnen pinnalta.  Kysymys Voiko rakennekynsiä laittaa jos on kynsisieni?  Vastaus  Rakennekynsiä ei voi laittaa kynsisienen päälle. Kynsi pitää aina hoitaa kuntoon ennen sitä.  Kysymys Rakennekynsi pilasi 

KeyboardInterrupt: 

In [15]:
for idx, extracted_entities in enumerate(answers):
    print("Original text:\n{}".format(dataset[idx]["text"]))
    print("Original id:\n{}".format(dataset[idx]["id"]))
    print("Extracted entities:")
    for entity in extracted_entities:
        print(entity)

Original text:
Olet täällä  31.1.2011  Kysymys  Terve, semmoista olisin kysynyt, että mitä tarkoittaa aivopoimut ja mihinkä se mahdollisesti vaikuttaa? Minun 4,5v tyttöni aivot on kuvattu ja lääkäri sanoi että mahdollisesti aivopoimuja voi olla liian vähän. Puheterapeutille oli lähete, mutta hänen mielestään artikuloi hyvin ja muutenkin ok..ei tarvitse sitä... Kaiken tämän ruljanssin takana on pieni pää tytölläni, jota on tutkittu kohta jo 3,5 vuotta ja selkoa ei ole tullut. Hänellä on todettu mikrokefalia, pään ympärys on tällä hetkellä 42,3 cm...että osaisitteko selittää niistä poimuista ja mihin ne mahdollisesti vaikuttaa? Lääkäri epäili että voi olla vaikeampaa oppia jotain asioita. Mutta olisi mukavaa jos voisitte jotain vastailla.  Vastaus  Hyvä kysyjä,  Noihin tutkimustuloksiin ei pidä tuijottaa. Pääasiahan on miten tyttärenne kehittyy. Jolleivät vanhemmat eivätkä edes puheterapeutti ole huomanneet mitään kuntoutettavaa, voitte mielestäni unohtaa löydökset. Jos jotain poikkeavaa

Original text:
Ketonen vastaa  moottorin merkkivalo kojelaudassa  Kysymys:  Minulla on ford mustang, v6, 3,8 litralla vm. 95. Siinä on semmoinen vika että moottorin merkkivalo syttyy joka toisella startilla. kahdessa pakoputkessa on neljä anturia yhteensä,( kaksi niistä vaihdettu uusiin) pitäisikö loputkin loputkin kaksi vaihtaa vai lieneekö vika jossain muualla? huom, joka toisella starttaamisella vikavalo syttyy! muuten moottori tuntuu toimivan.  Tapio Ketonen:  Periaatteessa vikavalon syttymiselle on aina jokin syy. En tunne Mustangia, joten surffailin hieman Jenkkilän keskustelupalstoilla. Mussessa on OBD I -diagnoosiliitäntä, sitä ei voi lukea Euroopassa yleisillä OBD II -lukijoilla. Joissakin autoissa on myös OBD II -liitäntä, mutta se ei ole kytketty toimivaksi. Autossa on myös Fordin EEC-IV-diagnoosiliitäntä.  Auton diagnoositietoihin pitäisi päästä käsiksi, jotta syy varoitusvalon syttymiseen selviäisi. Varmaankin sopivia diagnoosilaitteita löytyy joko Ford-korjamoilta tai sit

Original text:
Hakutulokset  Kysy allergiasta  Kysymys  Hei! Olen 55 v. nainen ja sain vasta vähän aikaa sitten astmadiagnoosin. Lääkkeinä on Alcesco ja Airomir. Minulla on astmakohtauksia aika usein, joskin lievähköjä. Onko tämä lääkitys hyvä vai voisiko olla parempia, jotka pitäisivät kohtaukset pois kokonaan? Onko normaalia, että astmakohtauksen aikana rupeaa jaloissa ja käsissä pistelemään, tikuttaa ja tulee muutenkin huono ja voimaton olo?  Vastaus  Alvesco on hengitettävä kortisoni, Airomir avaava lääke. Kyseessä on siis tavallinen astmalääkitys. Annoksen muuttamisesta tai mahdolisten yhdistelmävalmisteiden käytöstä päättää hoitava lääkäri. Pistely jaloissa ja käsissä ei kuulu ainakaan tavallisiin haittavaikutuksiin. Ne ovat joskus merkki ruoka- tms. allergiasta. Jos allergiatestit on tehty, oireiden mahdollista aiheuttajaa kannattaa seurailla muutama viikko. Siten mahdolliset jatkotestit on helpompi suunnitella, kun kaikkea ei kuitenkaan voi eikä kannata testata.
Original id:
6-

Original text:
Ketonen vastaa  Öljyjen sekoitus  Kysymys:  Voiko 15-20 vuotiaaseen japsimoottoriin (200-300tkm ajettuun) käyttää sekaisin täyssynteettisiä 5W-30, 5W-40 ja 5W-50 öljyjä? Eli kysymys tarkoittaa siis sitä, kun kannujen pohjille on jäänyt eri öljynvaihtojen yhteydessä tuotetta. Jotkut väittää, ettei voi sekoittaa ja itse olen ajanut viimeiset 5 vuotta tällä periaatteella enkä ole ongelmia huomannut. Mitä sanoo asiantuntija? Tiedän, että kysymykseeni on vastaus olemassa jo tuossa edellä, mutta itse en näe mitään syytä miksei voisi sekoittaa öljyjä keskenään.  Tapio Ketonen:  Periaatteessa moottoriöljyjen API-luokitukseen kuuluu vaatimus eri merkkisten öljyjen sekoitettavuudesta, joten eiköhän homma toimi myös teoriassa, kun kerran olet sitä soveltanut käytäntöönkin.  Noissa uusimmissa eurooppalaisissa moottoreissa pitää olla vähän tarkempana, niissä kun on tiukemmat vaatimukset öljylaaduille. Esimerkiksi Fordin moottorit vaativat ACEA A1/B1-luokitusten mukaista öljyä. Valmis

Original text:
Hakutulokset  Kysy allergiasta  Kysymys  Olen 58 v mies. Kaksi vuotta olen käyttänyt histeciapillereitä, kun en pärjää ilman. Parikin päivää ilman ja meinaan tukehtua yöllä limaan enkä voi nukkua. Aivastelen päivät niin että kallo on haljeta. Nenä on kipeä ja karstoittuu. Kun saan taas histecpillerin olo helpottaa ja olen oireeton. Haluaisin eroon pillereistä. Mitä teen asian kanssa? Olen siivooja ja töissä on paljon aineita ja pölyä.  Vastaus  Oireet ovat ilmeisesti ei-allergisia yliherkkyysoireita. Niin kauan homma on hallinnassa, kun Histec tai jokin muu antihistamiini auttaa. Niistähän ei ole pitkässäkään käytössä haittaa; niihin ei totu niin, että annosta pitäisi nostaa tai teho muutenkaan loppuisi. Antihistamiinit ovat lisäksi edullisia reseptillä ostettaessa.
Original id:
5-36362
Extracted entities:
{'entity_group': 'O', 'score': 0.9972861, 'word': 'Hakutulokset Kysy allergiasta Kysymys Olen 58 v mies. Kaksi vuotta olen käyttänyt histeciapillereitä, kun en pärjää 

Original text:
Kysymys: lattiavalun kuivuminen  Muut  Kyselin jo aikaisemmin keväällä muovimaton alta puskevan imelän hajun alkuperää, joka on nyt sitten paikallistettu kostean valun+primerin yhdistelmäksi. Eli n. 8 kk sitten valetusta maanvaraisesta 80mm laatasta on jätetty sementtiliima hiomatta (kunnolla) + päälle on vedetty praimeri pölyämisen estäjäksi (koska kyseessä laajennusosa, joka yhteydessä asuttuun osaan). Lattioihin vedettiin muovimatot oletetun kuivumisajan jälkeen (3 kk), jotka sitten n. kuukauden jälkeen asennuksesta poistettiin imelän hajaun takia. Lattiat olivat kuivumassa noin 3 kk paperilla suojattuna ja sen jälkeen pintakosteusmittareilla mitattuina olivat edelleenkin kosteita/märkiä. Nyt primerit+sementtiliimat on hiottu käsipelissä pois ja infrapunalämmittimet + kosteudenpoistaja ovat olleet viikon töissä ja tarkoituksena on tilata porareikämittaus (suhteellisen kosteuden mittaus) ensi viikolla. Kysymys kuuluu: onko todennäköistä, että valun kuivumattomuus johtu

Original text:
verta ulosteessa  25.11.2013  oireet  Kysyisin mieheni puolesta hänellä on n.2 vuoden ajan tullut ummetuksen yhteydessä verta pönttöön, veri on kirkasta ja sitä ei tule joka kerta vaan muutamien kuukausien välillä kun hänellä on ummetusta niin sillon ja paperiin tulee myös pyyhkiessä verta. Voiko olla syöpää vai peräpukama/haavauma ? Mieheni ei lähde lääkäriin. Ja olen huolissani  Hyvä kysyjä,  Yleisin syy kirkkaan verentuloon ulostuksen mukana ovat peräpukamat. Toinen mahdollinen verenvuodon aiheuttaja on peräaukon haavauma. Haavauma aiheuttaa myös kipua ulostuksen mukana. Ummetus lisää taipumusta kirkkaan verenvuodon tulemiseen ulostamisen yhteydessä. Peräpukamiin ja haavaumiin on olemassa hyvät ja helpot hoidot.  Ennen hoitojen aloittamista tulisi diagnoosi varmistaa, joten miehesi tulisi käydä oman lääkärin vastaanotolla. Vastaanottokäynnin yhteydessä arvioidaan myös, onko tarvetta paksusuolitähystyksen eli kolonoskopian tekemiselle. Usein tähystystä suositellaan, jo

KeyboardInterrupt: 

In [43]:
#from tqdm import tqdm

texts = dataset["text"]

In [63]:
questions = []
#answers = []
for idx, extracted_entities in enumerate(tqdm(tokens)):
    previous = ""
    question = ""
    answer = ""
  # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
  # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
    for entity in extracted_entities:
        if entity["entity_group"] == "QB":
            question = question + dataset[idx]["text"][entity["start"]:entity["end"]] #entity["word"]
            previous = "QB"
        # I should take into account that sometimes QI might be before QB but maybe this is more trustworthy
        elif previous == "QB" and entity["entity_group"] == "QI": #  or previous == "QI" and entity["entity_group"] == "QB"
            question = question + dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "QI"
        #elif entity["entity_group"] == "AB":
         #   answer = answer + dataset[idx]["text"][entity["start"]:entity["end"]]
          #  previous = "AB"
        #if previous == "AB" and entity["entity_group"] == "AI": # or previous == "AI" and entity["entity_group"] == "AB"
         #   answer = answer + dataset[idx]["text"][entity["start"]:entity["end"]]
          #  previous = "AI"
        elif entity["entity_group"] == "O":
            previous = "O"
    # if answer == "" or question == "":
    #   continue
    questions.append(question)
 #   answers.append(answer)

  0%|          | 0/31655 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [65]:
questions = []
for idx, extracted_entities in enumerate(tqdm(questions_data)):
    question2 = []
    previous = ""
    question = ""
    # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
    # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    for entity in extracted_entities:
        if entity["entity_group"] == "B":
            question = dataset[idx]["text"][entity["start"]:entity["end"]] #entity["word"]
            previous = "B"
        # I should take into account that sometimes QI might be before QB but maybe this is more trustworthy
        elif previous == "B" and entity["entity_group"] == "I": #  or previous == "I" and entity["entity_group"] == "B"
            question = question + " " + dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "I"
        elif entity["entity_group"] == "O":
            previous = "O"
        # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
        elif previous == "O" and entity["entity_group"] == "B":
            question2.append(question)
            question = dataset[idx]["text"][entity["start"]:entity["end"]]
    if len(question2):
        question2.append(question)
        questions.append(question2)
    else:
        questions.append(question)

  0%|          | 0/31655 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
answers = []
for i in range(len(questions_data)):
    answers.append("")

In [59]:
answers = []
for idx, extracted_entities in enumerate(tqdm(answers_data)):
    answer2 = []
    previous = ""
    answer = ""
    # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
    # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # now manually add space to at least fix some cases but now some words have a space in the middle
    for entity in extracted_entities:
        if entity["entity_group"] == "B":
            answer = dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "B"
        elif previous == "B" and entity["entity_group"] == "I": #  or previous == "I" and entity["entity_group"] == "B"
            answer = answer + " " + dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "I"
        elif entity["entity_group"] == "O":
            previous = "O"
        # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
        elif previous == "O" and entity["entity_group"] == "B":
            answer2.append(answer)
            answer = dataset[idx]["text"][entity["start"]:entity["end"]]
    if len(answer2):
        answer2.append(answer)
        answers.append(answer2)
    else:
        answers.append(answer)

  0%|          | 0/31655 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [66]:
for t, q, a in zip(texts, questions, answers):
    print("TEXT:", t)
    print("QUESTION:", q)
    print("ANSWER:", a)
    print("-------------")

TEXT: Olet täällä  31.1.2011  Kysymys  Terve, semmoista olisin kysynyt, että mitä tarkoittaa aivopoimut ja mihinkä se mahdollisesti vaikuttaa? Minun 4,5v tyttöni aivot on kuvattu ja lääkäri sanoi että mahdollisesti aivopoimuja voi olla liian vähän. Puheterapeutille oli lähete, mutta hänen mielestään artikuloi hyvin ja muutenkin ok..ei tarvitse sitä... Kaiken tämän ruljanssin takana on pieni pää tytölläni, jota on tutkittu kohta jo 3,5 vuotta ja selkoa ei ole tullut. Hänellä on todettu mikrokefalia, pään ympärys on tällä hetkellä 42,3 cm...että osaisitteko selittää niistä poimuista ja mihin ne mahdollisesti vaikuttaa? Lääkäri epäili että voi olla vaikeampaa oppia jotain asioita. Mutta olisi mukavaa jos voisitte jotain vastailla.  Vastaus  Hyvä kysyjä,  Noihin tutkimustuloksiin ei pidä tuijottaa. Pääasiahan on miten tyttärenne kehittyy. Jolleivät vanhemmat eivätkä edes puheterapeutti ole huomanneet mitään kuntoutettavaa, voitte mielestäni unohtaa löydökset. Jos jotain poikkeavaa ilmenee,